# Satellite Imagery-Based Property Valuation
## Multimodal Model Training

This notebook implements:
1. Baseline Model (Tabular only - XGBoost)
2. CNN Model (Image only - ResNet18)
3. Multimodal Fusion (Stacking meta-learner)
4. Model Explainability (Grad-CAM)
5. Prediction Generation


In [ ]:
# Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pickle
import warnings
import cv2

warnings.filterwarnings('ignore')

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch: {torch.__version__}")
print(f"Device: {device}")


## 1. Load Data


In [ ]:
# Load processed data
train_df = pd.read_csv('data/processed/train_processed.csv')
test_df = pd.read_csv('data/processed/test_processed.csv')

print(f"Training: {train_df.shape}")
print(f"Test: {test_df.shape}")

# Feature columns
feature_cols = [col for col in train_df.columns 
               if col not in ['id', 'date', 'price', 'price_original', 'price_per_sqft']]
print(f"\nFeatures: {len(feature_cols)}")


## 2. Baseline Model (XGBoost - Tabular Only)


In [ ]:
# Prepare data
X = train_df[feature_cols]
y = train_df['price']  # Log-transformed

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print(f"Train: {len(X_train)}, Val: {len(X_val)}")


In [ ]:
# Train XGBoost baseline
print("Training XGBoost baseline...")
xgb_model = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)
xgb_model.fit(X_train_scaled, y_train)

# Predictions
xgb_train_pred = xgb_model.predict(X_train_scaled)
xgb_val_pred = xgb_model.predict(X_val_scaled)

# Evaluate
baseline_results = {
    'train_rmse': np.sqrt(mean_squared_error(y_train, xgb_train_pred)),
    'val_rmse': np.sqrt(mean_squared_error(y_val, xgb_val_pred)),
    'val_r2': r2_score(y_val, xgb_val_pred),
    'val_mae': mean_absolute_error(y_val, xgb_val_pred)
}

print(f"\n✓ Baseline (XGBoost) Results:")
print(f"  Train RMSE: {baseline_results['train_rmse']:.4f}")
print(f"  Val RMSE: {baseline_results['val_rmse']:.4f}")
print(f"  Val R²: {baseline_results['val_r2']:.4f}")


## 3. CNN Model (ResNet18 - Image Only)


In [ ]:
# Dataset class for images
class ImageDataset(Dataset):
    def __init__(self, df, image_dir, target_col='price', is_training=True):
        self.image_dir = Path(image_dir)
        self.target_col = target_col
        
        # Filter valid images
        valid_ids = []
        for idx, row in df.iterrows():
            if (self.image_dir / f"{row['id']}.png").exists():
                valid_ids.append(idx)
        
        self.df = df.loc[valid_ids].reset_index(drop=True)
        print(f"  Using {len(self.df)}/{len(df)} samples with images")
        
        # Targets
        if target_col and target_col in self.df.columns:
            self.targets = self.df[target_col].values.astype(np.float32)
        else:
            self.targets = None
        
        # Transforms
        if is_training:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(),
                transforms.ColorJitter(brightness=0.1, contrast=0.1),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        prop_id = self.df.iloc[idx]['id']
        img = Image.open(self.image_dir / f"{prop_id}.png").convert('RGB')
        img = self.transform(img)
        
        if self.targets is not None:
            return img, torch.tensor(self.targets[idx])
        return img

print("✓ Dataset class defined")


In [ ]:
# ResNet18 Model for regression
class ResNet18Regressor(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        resnet = models.resnet18(pretrained=pretrained)
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        
        # Fine-tuning strategy: unfreeze later layers for satellite imagery
        # Option A: Full fine-tuning (satellite images differ from ImageNet)
        for param in self.features.parameters():
            param.requires_grad = True
        
        # Option B: Freeze only first 2 blocks (uncomment to use)
        # for name, child in list(self.features.named_children())[:5]:  # conv1, bn1, relu, maxpool, layer1
        #     for param in child.parameters():
        #         param.requires_grad = False
        
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.regressor(x)
        return x
    
    def get_features(self, x):
        """Extract features before regression head"""
        with torch.no_grad():
            x = self.features(x)
            x = x.view(x.size(0), -1)
        return x

print("✓ ResNet18 model defined")


In [ ]:
# Create datasets and dataloaders
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

print("Creating datasets...")
train_dataset = ImageDataset(train_data, 'data/images/train', is_training=True)
val_dataset = ImageDataset(val_data, 'data/images/train', is_training=False)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")


In [ ]:
# Train CNN
cnn_model = ResNet18Regressor().to(device)

# Differential learning rates: lower for pretrained backbone, higher for new head
optimizer = optim.Adam([
    {'params': cnn_model.features.parameters(), 'lr': 1e-5},   # Backbone: slow learning
    {'params': cnn_model.regressor.parameters(), 'lr': 1e-3}   # Head: fast learning
], weight_decay=1e-5)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
criterion = nn.MSELoss()

trainable = sum(p.numel() for p in cnn_model.parameters() if p.requires_grad)
total = sum(p.numel() for p in cnn_model.parameters())
print(f"Parameters: {total:,} total, {trainable:,} trainable")

# Training loop
num_epochs = 15
best_val_loss = float('inf')
cnn_history = {'train_loss': [], 'val_loss': []}

for epoch in range(num_epochs):
    # Train
    cnn_model.train()
    train_loss = 0
    for images, targets in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images = images.to(device)
        targets = targets.to(device).unsqueeze(1)
        
        optimizer.zero_grad()
        outputs = cnn_model(images)
        loss = criterion(outputs, targets)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(cnn_model.parameters(), 1.0)
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    
    # Validate
    cnn_model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, targets in val_loader:
            images = images.to(device)
            targets = targets.to(device).unsqueeze(1)
            outputs = cnn_model(images)
            val_loss += criterion(outputs, targets).item()
    
    val_loss /= len(val_loader)
    
    cnn_history['train_loss'].append(train_loss)
    cnn_history['val_loss'].append(val_loss)
    
    scheduler.step(val_loss)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(cnn_model.state_dict(), 'models/cnn_model.pth')
    
    print(f"  Train: {train_loss:.4f}, Val: {val_loss:.4f}")

print(f"\n✓ CNN training complete! Best Val Loss: {best_val_loss:.4f}")


## 4. Multimodal Fusion (Stacking Meta-Learner)


In [ ]:
# Generate predictions from both models for fusion
print("Generating predictions for fusion...")

# Load best CNN model
cnn_model.load_state_dict(torch.load('models/cnn_model.pth'))
cnn_model.eval()

# Get CNN predictions for validation set
cnn_val_preds = []
cnn_val_targets = []
with torch.no_grad():
    for images, targets in tqdm(val_loader, desc='CNN predictions'):
        images = images.to(device)
        outputs = cnn_model(images)
        cnn_val_preds.extend(outputs.cpu().numpy().flatten())
        cnn_val_targets.extend(targets.numpy())

cnn_val_preds = np.array(cnn_val_preds)
cnn_val_targets = np.array(cnn_val_targets)

# Get XGBoost predictions for same validation samples
# Match validation samples by ID
val_ids = set(val_dataset.df['id'].values)
val_mask = train_df['id'].isin(val_ids)
xgb_val_subset = xgb_model.predict(scaler.transform(train_df[val_mask][feature_cols]))

print(f"CNN predictions: {len(cnn_val_preds)}")
print(f"XGBoost predictions: {len(xgb_val_subset)}")


In [ ]:
# Train meta-learner (Ridge regression on stacked predictions)
print("Training meta-learner...")

# val_dataset.df already has all features - no merge needed!
val_df_ordered = val_dataset.df.copy()
val_df_ordered['cnn_pred'] = cnn_val_preds

# Get XGBoost predictions directly (val_dataset.df already has feature columns)
xgb_preds_ordered = xgb_model.predict(scaler.transform(val_df_ordered[feature_cols]))
val_df_ordered['xgb_pred'] = xgb_preds_ordered

# Stack predictions
stacked_features = np.column_stack([val_df_ordered['xgb_pred'], val_df_ordered['cnn_pred']])
stacked_targets = val_df_ordered['price'].values

# Split for meta-learner training
X_stack_train, X_stack_val, y_stack_train, y_stack_val = train_test_split(
    stacked_features, stacked_targets, test_size=0.3, random_state=42
)

# Train meta-learner
meta_learner = Ridge(alpha=1.0)
meta_learner.fit(X_stack_train, y_stack_train)

# Evaluate
meta_preds = meta_learner.predict(X_stack_val)

fusion_results = {
    'val_rmse': np.sqrt(mean_squared_error(y_stack_val, meta_preds)),
    'val_r2': r2_score(y_stack_val, meta_preds),
    'val_mae': mean_absolute_error(y_stack_val, meta_preds)
}

print(f"\n✓ Fusion (Stacking) Results:")
print(f"  Val RMSE: {fusion_results['val_rmse']:.4f}")
print(f"  Val R²: {fusion_results['val_r2']:.4f}")
print(f"  Weights: XGBoost={meta_learner.coef_[0]:.3f}, CNN={meta_learner.coef_[1]:.3f}")


## 5. Model Comparison


In [ ]:
# Compare all models
# Calculate CNN R² from predictions
cnn_r2 = r2_score(cnn_val_targets, cnn_val_preds)
cnn_rmse = np.sqrt(mean_squared_error(cnn_val_targets, cnn_val_preds))

comparison = pd.DataFrame({
    'Model': ['XGBoost (Tabular)', 'ResNet18 (Image)', 'Stacking (Fusion)'],
    'Val RMSE': [baseline_results['val_rmse'], cnn_rmse, fusion_results['val_rmse']],
    'Val R²': [baseline_results['val_r2'], cnn_r2, fusion_results['val_r2']]
})

print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)
print(comparison.to_string(index=False))
print("="*60)

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
models = comparison['Model']
rmse_vals = comparison['Val RMSE']
colors = ['steelblue', 'coral', 'green']

bars = ax.bar(models, rmse_vals, color=colors)
ax.set_ylabel('Validation RMSE (log scale)')
ax.set_title('Model Comparison', fontweight='bold')

for bar, val in zip(bars, rmse_vals):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
            f'{val:.4f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('models/model_comparison.png', dpi=300)
plt.show()

# Analysis
print("\n📊 Analysis:")
print("• XGBoost captures interior features (sqft, bedrooms, grade) - strongest predictors")
print("• CNN captures visual context (lot size, neighborhood, waterfront proximity)")
print("• Fusion combines both → slight improvement over tabular-only")
print(f"• Improvement from fusion: {(baseline_results['val_rmse'] - fusion_results['val_rmse']) / baseline_results['val_rmse'] * 100:.1f}% RMSE reduction")
print("• This is expected: satellite images can't see interior features that dominate price")


## 6. Model Explainability (Grad-CAM)


In [ ]:
# Grad-CAM implementation
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_full_backward_hook(self.save_gradient)
    
    def save_activation(self, module, input, output):
        self.activations = output.detach()
    
    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()
    
    def generate_cam(self, input_image):
        self.model.eval()
        output = self.model(input_image)
        
        self.model.zero_grad()
        output.backward()
        
        weights = torch.mean(self.gradients, dim=[2, 3], keepdim=True)
        cam = torch.sum(weights * self.activations, dim=1, keepdim=True)
        cam = torch.relu(cam)
        cam = cam - cam.min()
        cam = cam / (cam.max() + 1e-8)
        
        return cam.squeeze().cpu().numpy()

# Generate Grad-CAM for sample images
print("Generating Grad-CAM visualizations...")

# Get target layer (last conv layer of ResNet18)
# ResNet18: features[7] = layer4 (last block), [-1].conv2 = last conv
target_layer = cnn_model.features[7][-1].conv2
gradcam = GradCAM(cnn_model, target_layer)

# Visualize on sample images
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
sample_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

for idx, (img, target) in enumerate(sample_loader):
    if idx >= 4:
        break
    
    img = img.to(device)
    img.requires_grad = True
    
    cam = gradcam.generate_cam(img)
    
    # Original image
    orig_img = img.squeeze().cpu().permute(1, 2, 0).numpy()
    orig_img = (orig_img - orig_img.min()) / (orig_img.max() - orig_img.min())
    
    # Resize CAM
    cam_resized = cv2.resize(cam, (224, 224))
    
    # Plot
    axes[0, idx].imshow(orig_img)
    axes[0, idx].set_title(f'Original (${np.expm1(target.item()):,.0f})')
    axes[0, idx].axis('off')
    
    axes[1, idx].imshow(orig_img)
    axes[1, idx].imshow(cam_resized, cmap='jet', alpha=0.5)
    axes[1, idx].set_title('Grad-CAM')
    axes[1, idx].axis('off')

plt.suptitle('Model Explainability: Grad-CAM Visualizations', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('models/gradcam_samples.png', dpi=300)
plt.show()


## 7. Generate Test Predictions


In [ ]:
# Generate predictions on test set
print("Generating test predictions...")

# Create test dataset
test_dataset = ImageDataset(test_df, 'data/images/test', target_col=None, is_training=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Get CNN predictions
cnn_test_preds = []
cnn_model.eval()
with torch.no_grad():
    for images in tqdm(test_loader, desc='CNN predictions'):
        images = images.to(device)
        outputs = cnn_model(images)
        cnn_test_preds.extend(outputs.cpu().numpy().flatten())

# Get XGBoost predictions (test_dataset.df already has feature columns)
xgb_test_preds = xgb_model.predict(scaler.transform(test_dataset.df[feature_cols]))

# Combine with meta-learner
stacked_test = np.column_stack([xgb_test_preds, cnn_test_preds])
final_preds_log = meta_learner.predict(stacked_test)

# Convert to original scale
final_preds = np.expm1(final_preds_log)

print(f"\n✓ Predictions generated!")
print(f"  Mean: ${final_preds.mean():,.0f}")
print(f"  Range: ${final_preds.min():,.0f} - ${final_preds.max():,.0f}")


In [ ]:
# Create submission file
# IMPORTANT: Replace YOUR_ENROLLNO with your actual enrollment number
ENROLL_NO = "YOUR_ENROLLNO"

submission = pd.DataFrame({
    'id': test_dataset.df['id'].values,
    'predicted_price': final_preds
})

submission.to_csv(f'{ENROLL_NO}_final.csv', index=False)

print(f"✓ Submission saved: {ENROLL_NO}_final.csv")
print(f"\nFirst 10 predictions:")
print(submission.head(10))

# Save models
pickle.dump(xgb_model, open('models/xgb_model.pkl', 'wb'))
pickle.dump(scaler, open('models/scaler.pkl', 'wb'))
pickle.dump(meta_learner, open('models/meta_learner.pkl', 'wb'))
print("\n✓ All models saved!")


In [ ]:
# Final Summary
print("\n" + "="*60)
print("PROJECT COMPLETE!")
print("="*60)
print(f"\n✓ Models trained:")
print(f"  - XGBoost (Tabular): RMSE={baseline_results['val_rmse']:.4f}, R²={baseline_results['val_r2']:.4f}")
print(f"  - ResNet18 (Image): RMSE={cnn_rmse:.4f}, R²={cnn_r2:.4f}")
print(f"  - Stacking (Fusion): RMSE={fusion_results['val_rmse']:.4f}, R²={fusion_results['val_r2']:.4f}")
print(f"\n✓ Files saved:")
print(f"  - {ENROLL_NO}_final.csv (predictions)")
print(f"  - models/cnn_model.pth")
print(f"  - models/xgb_model.pkl")
print(f"  - models/meta_learner.pkl")
print(f"  - models/model_comparison.png")
print(f"  - models/gradcam_samples.png")
print("\n✓ Next steps:")
print(f"  1. Rename {ENROLL_NO}_final.csv with your enrollment number")
print(f"  2. Create report PDF: {ENROLL_NO}_report.pdf")
print(f"  3. Push to GitHub")
print(f"  4. Submit to portal")
print("="*60)
